<a href="https://colab.research.google.com/github/Kim-hyunbean/streamlit/blob/main/%EC%9B%B9%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIVLE스쿨 DX트랙 5차 미니프로젝트
## <span style="color:red;background-color:#fff5b1">[미션5] 대시보드 개발</span>

 * 119 구급대원 출동 일지 데이터를 바탕으로 대시보드를 구성해 보세요.
    * 당일 출동 이력
    * 일간/주간/월간 총 출동 건수
    * 일간/주간/월간 중증질환 비율
    * 일간/주간/월간 평균 이송 시간
    * 그 외 필요하다고 생각되는 정보

----------

### <span style="color:green;background-color:#fff5b1"> [5-1] 사전준비 </span>

 * Streamlit 사용을 위해 다음의 라이브러리 설치 필요
    * streamlit : 웹 시각화 프레임 워크
    * plotly : Interaction 시각화 프레임 워크


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 라이브러리 설치

!pip install streamlit
!pip install plotly
!pip install pyngrok

!pip install streamlit_folium
!pip install folium
!pip install geopy
!pip install haversine
!pip install joblib

!pip install xgboost
!pip install tensorflow
!pip install streamlit-calendar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.0/326.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00


-----------

### <span style="color:green;background-color:#fff5b1"> [5-2] streamlit으로 구현되는 대시보드 작성 </span>
 * 파일명은 dashboard.py로 함

In [19]:
#%%writefile app.py

# 라이브러리 불러오기

import pandas as pd
import numpy as np
import datetime
import joblib
from keras.models import load_model
from haversine import haversine
from urllib.parse import quote
import streamlit as st
from streamlit_folium import st_folium
import folium
import branca
from geopy.geocoders import Nominatim
import ssl

from streamlit_calendar import calendar


# 네이버 api
import json
import urllib
from urllib.request import Request, urlopen
import plotly.express as px

option = ''


def get_optimal_route(start, goal, option=option):
    client_id = 'fy9m99zzte'
    client_secret = 'gtOMi8OG2TbOuRhsfthH0dPV2WAEP8A2DAjTAW7x'

    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"

    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if res == 200:
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
    else:
        print('ERROR')


def results(results):
    lati_long = []
    japho = results['route']['traoptimal'][0]['path']
    for coord in japho:
        latitude, longitude = coord[1], coord[0]
        lati_long.append([latitude, longitude])
    return lati_long


# geocoding : 거리주소 -> 위도/경도 변환 함수
# Nominatim 파라미터 : user_agent = 'South Korea', timeout=None
# 리턴 변수(위도,경도) : lati, long
def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    lati = geo.latitude
    longit = geo.longitude
    return lati, longit

# -------------------------------------
#community_data = pd.read_csv('/content/drive/MyDrive/빅프csv/beebe.csv', encoding="cp949")
address = '서울특별시'
## 오늘 날짜
now_date = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=9)
now_date2 = datetime.datetime.strptime(now_date.strftime("%Y-%m-%d"), "%Y-%m-%d")

## 2024년 최소 날짜, 최대 날짜
first_date = pd.to_datetime("2024-01-01")
last_date = pd.to_datetime("2024-12-31")

st.set_page_config(layout="wide")

# tabs 만들기
tab1, tab2= st.tabs(['Bee119 신고 출동', '양봉장 밀원 분포지도'])

# tab1 내용물 구성하기
with tab1:

    # 제목 넣기
    st.markdown("## Bee119 신고 출동")

    # 시간 정보 가져오기
    now_date = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=9)


    # 정보 널기
    #st.markdown("#### 출동 정보")

    ## --------------------

    col110, col111, col112, col113 = st.columns([0.12, 0.28, 0.12, 0.28])
    with col110:
        st.info('신고일')
    with col111:
        input_date = st.date_input('날짜정보',label_visibility='collapsed')
    with col112:
        st.info('신고시간')
    with col113:
        input_time = st.time_input('시간정보',label_visibility='collapsed')


    ## -------------------------------------------------------------------------------------

    col120, col121, col122, col123 = st.columns([0.12, 0.28, 0.12, 0.28])
    with col120:
        st.info('사고위치')
    with col121:
        address1 = st.text_input('주소 텍스트 입력',value='서울특별시',label_visibility='collapsed')
    with col122:
        st.info('관할소방서')
    with col123:
        phonenum = st.text_input('소방서입력',value='소방서',label_visibility='collapsed')

    with st.form(key='tab1_first'):

        ### 버튼 생성
        if st.form_submit_button(label='위치조회'):

            #### 거리주소 -> 위도/경도 변환 함수 호출
            lati, longit = geocoding(address)

            display_df = pd.read_csv('/content/drive/MyDrive/빅프csv/양봉qhd.csv',encoding="cp949")
            display_df2 = pd.read_csv('/content/drive/MyDrive/빅프csv/꽃지도.csv',encoding="cp949")

            distance2 = []

            for idx1, row1 in display_df.iterrows():
                flower = '잡화,'
                for idx2, row2 in display_df2.iterrows():
                    rounded_distance = round(haversine((row2['위도'], row2['경도']), (row1['위도'], row1['경도']), unit='km'), 2)

                    if rounded_distance <= 2:
                        flower += row2['꽃'] + ','

                flowers = [i.strip() for i in flower.rstrip(',').split(',')]
                # 중복 제거
                unique_flowers = list(set(flowers))
                unique_flowers_join = ', '.join(unique_flowers)
                distance2.append(unique_flowers_join)

            display_df['근처 꽃'] = distance2

            distance = []
            patient = (lati, longit)

            for idx, row in display_df.iterrows():
                distance.append(round(haversine((row['위도'], row['경도']), patient, unit='km'), 2))

            display_df['거리'] = distance
            display_df['거리구분'] = pd.cut(display_df['거리'], bins=[-1, 2, 5, 10, 100],
                                 labels=['2km이내', '5km이내', '10km이내', '10km이상'])

            display_df = display_df[display_df.columns].sort_values(['거리구분', '거리'], ascending=[True, True])
            display_df.reset_index(drop=True, inplace=True)
            short_load = display_df[['위도','경도']]

            display_dff = display_df.drop(['위도','경도'],axis=1)

            if display_dff['거리'].iloc[0] <= 10 :
                st.markdown(f"### 예상되는 분봉 장소는 {display_dff['이름'].iloc[0]}입니다")

            #### 사고위치
            with st.expander("도시양봉맵", expanded=True):
                m = folium.Map(location=[lati,longit], zoom_start=12)
                icon = folium.Icon(color="red")
                html = """<!DOCTYPE html>
                    <html>
                        <table style="height: 126px; width: 330px;"> <tbody> <tr>
                            <td style="background-color: #FD5252;">
                            <div style="color: #ffffff;text-align:center;">벌 사고 발생 위치</div></td>
                            <td style="width: 230px;background-color: #C5C5C3;">{}</td>""".format(address)+"""</tr>
                        </tbody> </table> </html> """
                iframe = branca.element.IFrame(html=html, width=350, height=150)
                popup_text = folium.Popup(iframe,parse_html=True)
                folium.Marker(location=[lati , longit], popup=popup_text, tooltip="사고 발생 위치 : "+address, icon=icon).add_to(m)


                ###### 양봉지도

                for idx, row in display_df[:].iterrows():
                    html = """<!DOCTYPE html>
                    <html>
                        <table style="height: 126px; width: 330px;"> <tbody> <tr>
                            <td style="background-color: #F2952F;">
                            <div style="color: #ffffff;text-align:center;">양봉지명</div></td>
                            <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['이름'])+"""</tr>
                            <tr><td style="background-color: #F2952F;">
                            <div style="color: #ffffff;text-align:center;">주소</div></td>
                            <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['주소'])+"""</tr>
                            <tr><td style="background-color: #F2952F;">
                            <div style="color: #ffffff;text-align:center;">근처 꽃</div></td>
                            <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['근처 꽃'])+"""</tr>
                            <tr><td style="background-color: #F2952F;">
                            <div style="color: #ffffff;text-align:center;">연락처</div></td>
                            <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['연락처'])+"""</tr>
                        </tbody> </table> </html> """

                    iframe = branca.element.IFrame(html=html, width=360, height=200)
                    popup_text = folium.Popup(iframe,parse_html=True)
                    icon = folium.Icon(icon="forumbee",color="orange",prefix="fa")
                    folium.Marker(location=[row['위도'], row['경도']],
                                  popup=popup_text, tooltip=row['이름'], icon=icon).add_to(m)

                # 꽃지도
                for idx, row in display_df2[:].iterrows():
                    html = """<!DOCTYPE html>
                    <html>
                        <table style="height: 126px; width: 330px;"> <tbody> <tr>
                            <td style="background-color: #F2952F;">
                            <div style="color: #ffffff;text-align:center;">이름</div></td>
                            <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['이름'])+"""</tr>
                            <tr><td style="background-color: #F2952F;">
                            <div style="color: #ffffff;text-align:center;">주소</div></td>
                            <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['주소'])+"""</tr>
                            <tr><td style="background-color: #F2952F;">
                            <div style="color: #ffffff;text-align:center;">꽃</div></td>
                            <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['꽃'])+"""</tr>
                        </tbody> </table> </html> """

                    iframe = branca.element.IFrame(html=html, width=350, height=150)
                    popup_text = folium.Popup(iframe,parse_html=True)
                    icon = folium.Icon(icon="pagelines",color="pink",prefix="fa")
                    folium.Marker(location=[row['위도'], row['경도']],
                                  popup=popup_text, tooltip=row['이름'], icon=icon).add_to(m)

                start = (longit,lati)

                goal = (short_load.iloc[0]['경도'],short_load.iloc[0]['위도'])

                short = results(get_optimal_route(start,goal))
                folium.PolyLine(locations=short, color='#3098FE').add_to(m)


                st_folium(m, width=1000)
                st.dataframe(display_dff)

with tab2:
    st.markdown("## 양봉장 밀원 분포지도")

    display_df = pd.read_csv('/content/drive/MyDrive/빅프csv/양봉qhd.csv',encoding="cp949")
    display_df2 = pd.read_csv('/content/drive/MyDrive/빅프csv/꽃지도.csv',encoding="cp949")

    distance2 = []

    for idx1, row1 in display_df.iterrows():
        flower = '야생화,'
        for idx2, row2 in display_df2.iterrows():
            rounded_distance = round(haversine((row2['위도'], row2['경도']), (row1['위도'], row1['경도']), unit='km'), 2)

            if rounded_distance <= 2:
                flower += row2['꽃'] + ','

        flowers = [i.strip() for i in flower.rstrip(',').split(',')]
        # 중복 제거
        unique_flowers = list(set(flowers))
        unique_flowers_join = ', '.join(unique_flowers)
        distance2.append(unique_flowers_join)

    display_df['인근 꽃 정보'] = distance2

    display_dff = display_df.drop(['위도','경도'],axis=1)

    #### 사고위치
    with st.expander("꽃 지도", expanded=True):
        lati, longit = geocoding(address)
        m = folium.Map(location=[lati,longit], zoom_start=12)
        # 양봉지도
        for idx, row in display_df[:].iterrows():
            html = """<!DOCTYPE html>
                <html>
                    <table style="height: 126px; width: 330px;"> <tbody> <tr>
                        <td style="background-color: #F2952F;">
                        <div style="color: #ffffff;text-align:center;">양봉지명</div></td>
                        <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['이름'])+"""</tr>
                        <tr><td style="background-color: #F2952F;">
                        <div style="color: #ffffff;text-align:center;">주소</div></td>
                        <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['주소'])+"""</tr>
                        <tr><td style="background-color: #F2952F;">
                        <div style="color: #ffffff;text-align:center;">인근 꽃 정보</div></td>
                        <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['인근 꽃 정보'])+"""</tr>
                        <tr><td style="background-color: #F2952F;">
                        <div style="color: #ffffff;text-align:center;">연락처</div></td>
                        <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['연락처'])+"""</tr>
                    </tbody> </table> </html> """

            iframe = branca.element.IFrame(html=html, width=360, height=200)
            popup_text = folium.Popup(iframe,parse_html=True)
            icon = folium.Icon(icon="forumbee",color="orange",prefix="fa")
            folium.Marker(location=[row['위도'], row['경도']],
                            popup=popup_text, tooltip=row['이름'], icon=icon).add_to(m)

        # 꽃지도
        for idx, row in display_df2[:].iterrows():
            html = """<!DOCTYPE html>
                <html>
                    <table style="height: 126px; width: 330px;"> <tbody> <tr>
                        <td style="background-color: #F2952F;">
                        <div style="color: #ffffff;text-align:center;">이름</div></td>
                        <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['이름'])+"""</tr>
                        <tr><td style="background-color: #F2952F;">
                        <div style="color: #ffffff;text-align:center;">주소</div></td>
                        <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['주소'])+"""</tr>
                        <tr><td style="background-color: #F2952F;">
                        <div style="color: #ffffff;text-align:center;">꽃</div></td>
                        <td style="width: 230px;background-color: #FFBF00;text-align: center; color: white;">{}</td>""".format(row['꽃'])+"""</tr>
                    </tbody> </table> </html> """

            iframe = branca.element.IFrame(html=html, width=350, height=150)
            popup_text = folium.Popup(iframe,parse_html=True)
            icon = folium.Icon(icon="pagelines",color="pink",prefix="fa")
            folium.Marker(location=[row['위도'], row['경도']],
                            popup=popup_text, tooltip=row['이름'], icon=icon).add_to(m)


        st_folium(m, width=1000)
        st.dataframe(display_dff)


Overwriting app.py


In [ ]:
import streamlit as st
from streamlit_folium import st_folium
import folium

In [ ]:
help(folium.Icon(icon=''))

Help on Icon in module folium.map object:

class Icon(branca.element.MacroElement)
 |  Icon(color='blue', icon_color='white', icon='info-sign', angle=0, prefix='glyphicon', **kwargs)
 |  
 |  Creates an Icon object that will be rendered
 |  using Leaflet.awesome-markers.
 |  
 |  Parameters
 |  ----------
 |  color : str, default 'blue'
 |      The color of the marker. You can use:
 |  
 |          ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 |           'gray', 'black', 'lightgray']
 |  
 |  icon_color : str, default 'white'
 |      The color of the drawing on the marker. You can use colors above,
 |      or an html color code.
 |  icon : str, default 'info-sign'
 |      The name of the marker sign.
 |      See Font-Awesome website to choose yours.
 |      Warning : depending on the icon you choose you may need to adapt
 |      the 

----------

### <span style="color:green;background-color:#fff5b1"> [4-3] 대시보드 확인하기 </span>

* dispatch_log.py 파일이 있는 디렉토리로 이동
* 'Shift + 마우스 우클릭' -> '터미널에서 열기' 클릭
* -> '터미널에서 열기'  메뉴가 없으면 '여기에 PowerShell 창 열기(S)' 클릭
* 아래의 streamlit 실행( ! 빼고 실행)
* training.py를 수정후 브라우저의 새로고침을 누르면 프로토타입에 자동 변경됨
* 새로고침을 통해 변경사항 확인

In [ ]:
!ngrok authtoken 2XbOiYEFkz2390JjiiMorbfPEzW_5YUzWNV6aBwKPSsALVZiy

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

!streamlit run app.py&>/dev/null&

# ngrok을 사용하여 Streamlit 앱을 외부로 노출
public_url = ngrok.connect(addr='8501')
public_url

<NgrokTunnel: "https://6a2c-34-145-215-2.ngrok-free.app" -> "http://localhost:8501">

In [ ]:
ngrok.kill()

## <span style="color:red;background-color:#fff5b1">[도전 미션] 프로토타입 + 대시보드 합치기</span>

  * 미션4 에서 만든 프로토타입과 미션5에서 만든 대시보드를 하나의 파일로 합쳐서 최종 솔루션 프로토타입을 완성 하세요.
  * 최종 솔루션 프로토타입의 파일명은 app.py 입니다.
  * app.py 가 완성되면 Streamlit으로 실행하여 프로토타입과 대시보드가 잘 동작 하는지 체크해 주세요.

In [ ]:
'''

# session_state에 키 값 체크, 없으면 초기값 할당
if "pokemons" not in st.session_state:
    st.session_state.pokemons = pd.DataFrame(columns=["작성자", "제목", "내용", "작성일"])

def write_post(author, title, content):

    # 새로운 행을 생성하고 데이터프레임에 추가합니다.
    now_date = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=9)
    formatted_date = now_date.strftime("%Y-%m-%d")
    new_post = {"작성자": author, "제목": title, "내용": content, "작성일": formatted_date}

    # 기존 데이터프레임과 새로운 행을 결합합니다.
    st.session_state.pokemons = st.session_state.pokemons.append(new_post, ignore_index=True)

    # 성공 메시지를 표시합니다.
    st.success("게시글이 성공적으로 작성되었습니다.")

def display_posts():
    if st.session_state.pokemons.empty:
        st.info("작성된 게시글이 없습니다.")
    else:
        st.table(st.session_state.pokemons)


    mode = st.selectbox(
        "달력 설정:",
        (
            "daygrid",
            "list",
            "multimonth",
        ),
    )

    events = [
        {
            "title": "Event 1",
            "color": "#FF6C6C",
            "start": "2023-01-03",
            "end": "2023-01-05",
            "resourceId": "a",
        },
        {
            "title": "Event 2",
            "color": "#FFBD45",
            "start": "2023-01-01",
            "end": "2023-01-10",
            "resourceId": "b",
        },
        {
            "title": "Event 3",
            "color": "#FF4B4B",
            "start": "2023-01-20",
            "end": "2023-01-20",
            "resourceId": "c",
        },
        {
            "title": "Event 4",
            "color": "#FF6C6C",
            "start": "2023-01-23",
            "end": "2023-01-25",
            "resourceId": "d",
        },
        {
            "title": "Event 5",
            "color": "#FFBD45",
            "start": "2023-01-29",
            "end": "2023-01-30",
            "resourceId": "e",
        },
        {
            "title": "Event 6",
            "color": "#FF4B4B",
            "start": "2023-01-28",
            "end": "2023-01-20",
            "resourceId": "f",
        },
        {
            "title": "Event 7",
            "color": "#FF4B4B",
            "start": "2023-01-01T08:30:00",
            "end": "2023-01-01T10:30:00",
            "resourceId": "a",
        },
        {
            "title": "Event 8",
            "color": "#3D9DF3",
            "start": "2023-01-01T07:30:00",
            "end": "2023-01-01T10:30:00",
            "resourceId": "b",
        },
        {
            "title": "Event 9",
            "color": "#3DD56D",
            "start": "2023-01-02T10:40:00",
            "end": "2023-01-02T12:30:00",
            "resourceId": "c",
        },
        {
            "title": "Event 10",
            "color": "#FF4B4B",
            "start": "2023-01-15T08:30:00",
            "end": "2023-01-15T10:30:00",
            "resourceId": "d",
        },
        {
            "title": "Event 11",
            "color": "#3DD56D",
            "start": "2023-01-15T07:30:00",
            "end": "2023-01-15T10:30:00",
            "resourceId": "e",
        },
        {
            "title": "Event 12",
            "color": "#3D9DF3",
            "start": "2023-01-21T10:40:00",
            "end": "2023-01-21T12:30:00",
            "resourceId": "f",
        },
        {
            "title": "Event 13",
            "color": "#FF4B4B",
            "start": "2023-01-17T08:30:00",
            "end": "2023-01-17T10:30:00",
            "resourceId": "a",
        },
        {
            "title": "Event 14",
            "color": "#3D9DF3",
            "start": "2023-01-17T09:30:00",
            "end": "2023-01-17T11:30:00",
            "resourceId": "b",
        },
        {
            "title": "Event 15",
            "color": "#3DD56D",
            "start": "2023-01-17T10:30:00",
            "end": "2023-01-17T12:30:00",
            "resourceId": "c",
        },
        {
            "title": "Event 16",
            "color": "#FF6C6C",
            "start": "2023-01-17T13:30:00",
            "end": "2023-01-17T14:30:00",
            "resourceId": "d",
        },
        {
            "title": "Event 17",
            "color": "#FFBD45",
            "start": "2023-01-17T15:30:00",
            "end": "2023-01-17T16:30:00",
            "resourceId": "e",
        },
    ]
    calendar_resources = [
        {"id": "a", "building": "Building A", "title": "Room A"},
        {"id": "b", "building": "Building A", "title": "Room B"},
        {"id": "c", "building": "Building B", "title": "Room C"},
        {"id": "d", "building": "Building B", "title": "Room D"},
        {"id": "e", "building": "Building C", "title": "Room E"},
        {"id": "f", "building": "Building C", "title": "Room F"},
    ]

    calendar_options = {
        "editable": "true",
        "navLinks": "true",
        "resources": calendar_resources,
        "selectable": "true",
    }

    if mode == "daygrid":
        calendar_options = {
            **calendar_options,
            "headerToolbar": {
                "left": "today prev,next",
                "center": "title",
                "right": "dayGridMonth",
            },
            "initialDate": "2023-01-01",
            "initialView": "dayGridMonth",
        }
    elif mode == "list":
        calendar_options = {
            **calendar_options,
            "initialDate": "2023-01-01",
            "initialView": "listMonth",
        }
    elif mode == "multimonth":
        calendar_options = {
            **calendar_options,
            "initialView": "multiMonthYear",
        }

    state = calendar(
        events=st.session_state.get("events", events),
        options=calendar_options,
        custom_css="""
        .fc-event-past {
            opacity: 0.8;
        }
        .fc-event-time {
            font-style: italic;
        }
        .fc-event-title {
            font-weight: 700;
        }
        .fc-toolbar-title {
            font-size: 2rem;
        }
        """,
        key=mode,
    )

    if state.get("eventsSet") is not None:
        st.session_state["events"] = state["eventsSet"]

    st.write(state)

with tab3:

    # 탭을 추가
    tab01, tab02, tab03 = st.tabs(["홈", "게시판", "글 작성"])

    with tab01:
        # 홈 탭의 내용
        st.header("커뮤니티 홈")
        st.write("이곳에 커뮤니티의 홈 내용을 작성하세요.")

    with tab02:
        # 게시판 탭의 내용
        st.header("커뮤니티 게시판")
        display_posts()

    with tab03:
        # 글 작성 탭의 내용
        selcom = st.selectbox('판단',('커뮤니티','교육 예약'),label_visibility='collapsed')
        if selcom == '교육 예약':
            col110, col111 = st.columns([0.1, 0.9])
            with col110:
                st.info('날짜')
            with col111:
                input_date = st.date_input('날짜정보',label_visibility='collapsed')

            col112, col113 = st.columns([0.1, 0.9])
            with col112:
                st.info('이름')
            with col113:
                author = st.text_input("이름",label_visibility='collapsed')

            col114, col115 = st.columns([0.1, 0.9])
            with col114:
                st.info('연락처')
            with col115:
                author = st.text_input("연락처",label_visibility='collapsed')

            if st.button("신청"):
                st.success('예약이 완료되었습니다')
        else:
            author = st.text_input("작성자", "익명")
            title = st.text_input("제목")
            content = st.text_area("내용")
            if st.button("글 작성"):
                write_post(author, title, content)
                display_posts()'''

----------

# 수고하셨습니다.

----------